In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
def find_unique_most_recent(df, id_name):
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date', ascending = False)
    df = df.reset_index(drop = True)

    u, indices = np.unique(df[id_name].values, return_index = True) # we are trying to take the most recent rating
    indices.sort()
    mask_unique = df.index.isin(indices)
    df = df[mask_unique]
    return df

In [3]:
def filter_into_pl_and_non_pl():
    matches = pd.read_csv('Match.csv')
    mask = matches['country_id'] == 1729 # only PL
    matches_pl = matches[mask]
    matches_non_pl = matches[~mask]
    del matches
    return matches_pl, matches_non_pl
    '''
    teams = pd.read_csv('Team.csv')
    pl_teams = np.unique(matches_pl[['home_team_api_id', 'away_team_api_id']].values)
    mask = teams['team_api_id'].isin(pl_teams)
    teams_pl = teams[mask]
    teams_non_pl = teams[~mask]
    
    teams_attr = pd.read_csv('Team_Attributes.csv')
    pl_teams_attr = teams_pl['team_api_id']
    mask = teams_attr['team_api_id'].isin(pl_teams_attr)
    teams_attr_pl = teams_attr[mask]
    teams_attr_non_pl = teams_attr[~mask]
    ta_pl = find_unique_most_recent(teams_attr_pl, 'team_api_id')
    ta_non_pl = find_unique_most_recent(teams_attr_non_pl, 'team_api_id')
    
    players = pd.read_csv('Player.csv')
    '''

In [18]:
def construct_input(matches, player_attr, columns_keep):
    match_ids = matches['match_api_id']
    goal_dif = matches['home_team_goal'].sub(matches['away_team_goal'])
    goal_dif[goal_dif < 0] = -1
    goal_dif[goal_dif > 0] = 1
    positions = ['home_player_1',
                 'home_player_2',
                 'home_player_3',
                 'home_player_4',
                 'home_player_5',
                 'home_player_6',
                 'home_player_7',
                 'home_player_8',
                 'home_player_9',
                 'home_player_10',
                 'home_player_11',
                 'away_player_1',
                 'away_player_2',
                 'away_player_3',
                 'away_player_4',
                 'away_player_5',
                 'away_player_6',
                 'away_player_7',
                 'away_player_8',
                 'away_player_9',
                 'away_player_10',
                 'away_player_11']
    columns = []
    for pos in positions:
        arr = [pos + '_' + column_name for column_name in columns_keep]
        columns.append(arr)
    
    player_series = [matches[pos] for pos in positions]
    columns_len = len(positions) * (len(player_attr.columns) - 1)
    column_attrs = []
    for i, pos in enumerate(positions):
        attr = player_attr.loc[player_series[i].values].reset_index(drop = True)
        for j, column in enumerate(attr):
            column_attrs.append(attr[column].rename(columns[i][j]))
    
    x = pd.concat(column_attrs, axis = 1)
    y = goal_dif
    return x, y

In [5]:
def split_matches_data(m_pl, m_npl):
    pl_tr = 0.2
    pl_cv = 0.4
    pl_te = 0.4
    m_pl_tr = m_pl.sample(frac = pl_tr)
    m_pl = m_pl.drop(m_pl_tr.index)
    m_pl_cv = m_pl.sample(frac = (pl_cv / (pl_cv + pl_te)))
    m_pl = m_pl.drop(m_pl_cv.index)
    m_pl_te = m_pl
    
    m_tr = m_npl.append(m_pl_tr).sample(frac = 1)
    
    return m_tr, m_pl_cv, m_pl_te

In [6]:
def get_player_attr():
    df = find_unique_most_recent(pd.read_csv('Player_Attributes.csv'), 'player_api_id')
    df = df.set_index('player_api_id')
    columns_keep = ['overall_rating',
                    'potential',
                    'crossing',
                    'finishing',
                    'heading_accuracy',
                    'short_passing',
                    'volleys',
                    'dribbling',
                    'curve',
                    'free_kick_accuracy',
                    'long_passing',
                    'ball_control',
                    'acceleration',
                    'sprint_speed',
                    'agility',
                    'reactions',
                    'balance',
                    'shot_power',
                    'jumping',
                    'stamina',
                    'strength',
                    'long_shots',
                    'aggression',
                    'interceptions',
                    'positioning',
                    'vision',
                    'penalties',
                    'marking',
                    'standing_tackle',
                    'sliding_tackle',
                    'gk_diving',
                    'gk_handling',
                    'gk_kicking',
                    'gk_positioning',
                    'gk_reflexes']
    temp_df = df[columns_keep]
    return temp_df, columns_keep

In [20]:
def load_data_all():
    matches_pl, matches_non_pl = filter_into_pl_and_non_pl()
    player_attr, columns_keep = get_player_attr()
    m_tr, m_cv, m_te = split_matches_data(matches_pl, matches_non_pl)
    x_tr, y_tr = construct_input(m_tr, player_attr, columns_keep)
    x_cv, y_cv = construct_input(m_cv, player_attr, columns_keep)
    x_te, y_te = construct_input(m_te, player_attr, columns_keep)
    
    min_max_scaler = preprocessing.MinMaxScaler()
    x_tr = pd.DataFrame(min_max_scaler.fit_transform(x_tr), index = m_tr['match_api_id'], columns = x_tr.columns)
    x_cv = pd.DataFrame(min_max_scaler.fit_transform(x_cv), index = m_cv['match_api_id'], columns = x_cv.columns)
    x_te = pd.DataFrame(min_max_scaler.fit_transform(x_te), index = m_te['match_api_id'], columns = x_te.columns)

    return x_tr, y_tr, x_cv, y_cv, x_te, y_te

In [16]:
df = pd.read_csv('Match.csv')

In [ ]:
df['home_player_1'].isnull().sum()

In [ ]:
player_attr, columns_keep = get_player_attr()

In [ ]:

positions = ['home_player_1',
             'home_player_2',
             'home_player_3',
             'home_player_4',
             'home_player_5',
             'home_player_6',
             'home_player_7',
             'home_player_8',
             'home_player_9',
             'home_player_10',
             'home_player_11',
             'away_player_1',
             'away_player_2',
             'away_player_3',
             'away_player_4',
             'away_player_5',
             'away_player_6',
             'away_player_7',
             'away_player_8',
             'away_player_9',
             'away_player_10',
             'away_player_11']
columns = []
for pos in positions:
    columns = columns + [pos + '_' + column_name for column_name in columns_keep]
x_cv.columns = columns
x_cv.columns
x_cv

In [19]:
x_tr, y_tr, x_cv, y_cv, x_te, y_te = load_data_all()

Index(['home_player_1_overall_rating', 'home_player_1_potential',
       'home_player_1_crossing', 'home_player_1_finishing',
       'home_player_1_heading_accuracy', 'home_player_1_short_passing',
       'home_player_1_volleys', 'home_player_1_dribbling',
       'home_player_1_curve', 'home_player_1_free_kick_accuracy',
       ...
       'away_player_11_vision', 'away_player_11_penalties',
       'away_player_11_marking', 'away_player_11_standing_tackle',
       'away_player_11_sliding_tackle', 'away_player_11_gk_diving',
       'away_player_11_gk_handling', 'away_player_11_gk_kicking',
       'away_player_11_gk_positioning', 'away_player_11_gk_reflexes'],
      dtype='object', length=770)


In [24]:
x_tr.to_csv('x_train.csv', index_label = False)
y_tr.to_csv('y_train.csv', index_label = False)
x_cv.to_csv('x_val.csv', index_label = False)
y_cv.to_csv('y_val.csv', index_label = False)
x_te.to_csv('x_test.csv', index_label = False)
y_te.to_csv('y_test.csv', index_label = False)